### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    tm = today - relativedelta(days=1)
    limite = tm.replace(day=15)
else:
    tm = today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day)
    limite = tm.replace(day=15)

### GOOGLE SHEETS

In [4]:
# Partners PEYA
sheet_id = '1HmAvHYbJJa3JyRTgRJGKXataQB_TvFwNzyo8a6qXd2o'
wks_name = 'Partners PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
partners = sheet.sheet_to_df(index=0,header_rows=1)

In [5]:
# Stats PEYA
sheet_id = '1PO5kGLgz5Xn_WNCQQB0OPcKCl7KeS7kMrkJaGxafqmY'
wks_name = 'Stats PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
stats = sheet.sheet_to_df(index=0,header_rows=1)

In [6]:
# RCP
sheet_id = '1JNywQTVzEQKRwqrJRkpzjiXx5Ly-FldtBMfeSYHuL7w'
wks_name = 'RCP'
sheet = Spread(sheet_id, wks_name, config=cred)
rcp = sheet.sheet_to_df(index=0,header_rows=1)

### FUNCIONES

In [7]:
def clasificacion(i):
    if i['Fecha_Cierre'] != '-':
        cierre = datetime.datetime.strptime(i['Fecha_Cierre'],'%Y-%m-%d').date()
        if cierre > limite:
            val = 'New Online'
        else:
            val = 'Activa'
    else:
        if i['Dias_Online LM'] > 0 and i['Confirmed LM'] == 0:
            val = 'RCP Next Month'
        else:
            val = 'Activa'
    return val

### TRABAJO

#### TRABAJO ZOMBIES

In [8]:
# Hago lista de RCP
rcp = [x.upper() for x in rcp[rcp.columns[0]]]

In [9]:
# Doy formato a las columnas de Stats
cols = [i for i in stats.columns if i not in ['Id','Fecha_Cierre','Online','Account_Owner']]
stats[cols] = stats[cols].astype(float)

In [10]:
# Filtro las columnas a usar
cols_partners = ['Id','Grid','Online','Account_Owner','Area','Feudo','Reino','KAM','Business','Concept']
cols_stats = ['Id','Fecha_Cierre','Dias_Online TM','Dias_Online LM','Confirmed TM','Confirmed LM']
partners = partners[cols_partners].copy()
stats = stats[cols_stats].copy()

In [11]:
# Doy formato a las columnas
partners['Id'] = partners['Id'].astype(int)
stats['Id'] = stats['Id'].astype(int)
# Hago el merge
final = partners.merge(stats,how='left')
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [12]:
# Coloco la Clasificacion
final['Clasificacion'] = final.apply(clasificacion,axis=1)
# Marco los Zombies
final['Zombie Empresa'] = final.apply(lambda x: 'Si' if x['Confirmed TM'] == 0 and x['Dias_Online TM'] > 0 else 'No',axis=1)
final['Zombie Account'] = final.apply(lambda x: 'Si' if x['Zombie Empresa'] == 'Si' and x['Online'] == 'Si' else 'No',axis=1)

In [13]:
# Ordeno las columnas
cols = ['Id','Grid','Online','Fecha_Cierre','Dias_Online LM','Dias_Online TM','Confirmed LM','Confirmed TM','Clasificacion',
        'Zombie Empresa','Zombie Account']
base_zombies = final[cols].copy()
# Ordeno segun Id
base_zombies.sort_values(by=['Id'],inplace=True)

#### TRABAJO OBJETIVO ZOMBIES

In [14]:
# Marco las RCP
final['Account_Owner'] = final['Account_Owner'].str.upper()
final['RCP'] = final['Account_Owner'].apply(lambda x: 'Si' if x in rcp else 'No')

In [15]:
# Coloco el Business Final
final['Business Final'] = final['Business'].apply(lambda x: 'Restaurant' if x == 'Restaurant' else 'NV')
# Creo una columna de base para contas las cuentas
final['Carteras'] = final['Dias_Online TM'].apply(lambda x: 1 if x > 0 else 0)

In [16]:
# Hago una PT para las Carteras
carteras_cm = pd.pivot_table(final,index=['Reino','Feudo','KAM','Concept','RCP','Business Final','Clasificacion'],values=['Carteras'],aggfunc='sum',fill_value=0).reset_index()
carteras_am = pd.pivot_table(final,index=['Account_Owner','Clasificacion'],values=['Carteras'],aggfunc='sum',fill_value=0).reset_index()

In [17]:
# Separo los Zombies
zombies = final[final['Zombie Empresa'] == 'Si'].copy()
# Ordeno las columnas
cols = ['Grid','Id','Area','Feudo','Reino','KAM','Business','Account_Owner','Concept','Online','Fecha_Cierre',
        'Dias_Online LM','Dias_Online TM','Confirmed LM','Confirmed TM','Clasificacion','Zombie Empresa','Zombie Account',
        'RCP','Carteras','Business Final']
zombies = zombies[cols].copy()

### CARGA

In [18]:
# Carga Zombies PEYA
sheet_id = '11PbmF7vo_A1wfcSIY00iwy6lU04LjyWLjZ4SPebyqbA'
wks_name = 'Zombies PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(base_zombies, index=False, sheet=wks_name, replace=True)

In [21]:
# Carga Cartera CM
sheet_id = '1CHBlR8cFKRy-ycRdf_NAcTvGhA9TgfaKytJSJENM5xk'
wks_name = 'Cuentas RM-CM'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(carteras_cm, index=False, sheet=wks_name, replace=True)

In [22]:
# Carga Cartera AM
sheet_id = '1CHBlR8cFKRy-ycRdf_NAcTvGhA9TgfaKytJSJENM5xk'
wks_name = 'Cuentas Accounts'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(carteras_am, index=False, sheet=wks_name, replace=True)

In [23]:
# Carga Zombies
sheet_id = '1CHBlR8cFKRy-ycRdf_NAcTvGhA9TgfaKytJSJENM5xk'
wks_name = 'Zombies'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(zombies, index=False, sheet=wks_name, replace=True)